In [2]:
import pandas as pd

base_path = 'ml-100k/'

# 1. Load ratings (u.data: tab-separated userId | movieId | rating | timestamp)
ratings_cols = ['userId','movieId','rating','timestamp']
ratings = pd.read_csv(
    f'{base_path}u.data', 
    sep='\t', 
    names=ratings_cols, 
    engine='python'
)

# 2. Load movies (u.item: pipe-separated; first 5 cols + 19 genre flags)
item_cols = ['movieId','title','release_date','video_release_date','IMDb_URL']
genre_cols = [
    'unknown','Action','Adventure','Animation',"Children's",'Comedy','Crime',
    'Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
    'Romance','Sci-Fi','Thriller','War','Western'
]
movies = pd.read_csv(
    f'{base_path}u.item',
    sep='|',
    names=item_cols + genre_cols,
    encoding='latin-1',
    engine='python'
)

# 3. Quick sanity-check
print(ratings.head())
print(movies.head())


   userId  movieId  rating  timestamp
0     196      242       3  881250949
1     186      302       3  891717742
2      22      377       1  878887116
3     244       51       2  880606923
4     166      346       1  886397596
   movieId              title release_date  video_release_date  \
0        1   Toy Story (1995)  01-Jan-1995                 NaN   
1        2   GoldenEye (1995)  01-Jan-1995                 NaN   
2        3  Four Rooms (1995)  01-Jan-1995                 NaN   
3        4  Get Shorty (1995)  01-Jan-1995                 NaN   
4        5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://